In [1]:
import os
import pandas as pd 
from dotenv import load_dotenv
import asyncio
import nest_asyncio
nest_asyncio.apply()


from glob import glob
from langchain_openai import AzureChatOpenAI,AzureOpenAIEmbeddings
from datasets import load_dataset, Dataset,DatasetDict
from ragas import evaluate
from ragas.metrics import context_precision,answer_relevancy,faithfulness,context_recall,answer_similarity,answer_correctness

from project_autodesk.autodesk.rag_process.response_process import llm_response
from project_autodesk.autodesk.rag_process.retrieval_clients import azure_document_retriever



load_dotenv(override=True)

c:\Users\User\anaconda3\envs\test2env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
context = {"overrides":{"retrieval_mode": "hybrid",
                        "semantic_ranker":True,
                        "semantic_captions":True,
                        "top":3}}

In [3]:
def merge_contents(data):

    merged= [[doc.page_content] for doc in data]

    return merged


def make_evaluation_data(test_data):

    data =  DatasetDict({
        "eval" : Dataset.from_dict(test_data)
    })

    return data 


async def generate_evaluation_data(data:pd.DataFrame):
    

    for idx in range(len(data)):

        q= data.loc[idx,"question"]
        documents = await azure_document_retriever.run(q,context)

        data.loc[idx,"contexts"]=merge_contents(documents)

        response = await(llm_response(q))

        data.loc[idx,"answer"]= response["answer"]


    return data


In [4]:
#Load in User defined questions as csv

qna_data = pd.read_csv("dev_qna_data.csv")#,encoding='cp1252')

In [5]:
qna_data

,question,ground_truth
0,How do we authenticate User licenses?,"User licenses, based on the customers Autodesk..."
1,How do we authenticate Standalone licenses?,Standalone licenses are tied to specific devic...
2,How many types of licenses do adminstrators ma...,"Administrators manage three types of licenses,..."
3,What is Fusion 360 for Manufacturing?,Autodesk Fusion 360 for Manufacturing is an in...
4,What are Advanced CAM capabilities?,Unlock advanced tools for multi-axis CNC machi...
5,Who uses the Fusion 360 for Manufacturing offe...,Fusion 360 for Manufacturing is built for orga...
6,How does Fusion 360 for Manufacturing differ t...,Autodesk offers a range of Fusion 360 extensio...
7,Does Autodesk offer free software for students...,"If you are a student, educator, or an IT or la..."
8,What are the Flexnet feature codes,Feature codes come in three types: product-spe...
9,What are the uses of Revit + AutoCAD + Civil 3D,"USE IT FOR: Building design, Infrastructure de..."


In [6]:
#Run the question and ground_truth dataframe to retrieve contents and answers.

res = asyncio.run(generate_evaluation_data((qna_data)))

{'https://samplehtmlstore.blob.core.windows.net/autodesk/adsk-3b5f10815afc4d8c9311bf594d3c206b.html': '\nFeature codes\n\nCascading sequences for Autodesk products\n\nInstallation for administrators\n\nOverview of license types\n\nAdministrators manage three types of licenses, according to the subscription type.\n\nUser licenses\n\nStandalone (device\n\n-\n\nbased) licenses\n\nNetwork licenses\n\nEach license type is authenticated differently.\n\nUser licenses, based on the customer’s Autodesk ID, are automatically authenticated when the customer first runs a product after signing in to Autodesk Account.\n\nStandalone licenses are tied to specific devices and use serial numbers. When customers first run the product, standalone licenses are authenticated either silently by Autodesk servers or when customers are prompted to enter a serial number.\n\nNetwork licenses, managed with Network License Manager (NLM) software, are authenticated by preconfigured network license servers. Sometimes

In [7]:
# Format Evaluation data
eval_data = make_evaluation_data(res)

In [8]:

#OpenAI Credentials
open_ai_service=os.environ.get("OPENAI_API_ENDPOINT")
open_ai_key=os.environ.get("OPEN_AI_KEY")
open_ai_deployment=os.environ.get("OPENAI_AI_DEPLOYMENT_NAME")
open_ai_model_name=os.environ.get("OPEN_AI_MODEL_NAME")
open_ai_host=os.environ.get("OPEN_AI_HOST")

#initialize LLM
azure_openai_client = AzureChatOpenAI(azure_endpoint = f"https://{open_ai_service}.openai.azure.com",
                      azure_deployment =open_ai_deployment,
                      model=open_ai_model_name,
                      api_version =  "2023-03-15-preview",
                      api_key = open_ai_key)



azure_openai_embedding_client = AzureOpenAIEmbeddings(azure_endpoint = f"https://{open_ai_service}.openai.azure.com",
                      azure_deployment ="text-embedding-ada-002",
                      model="text-embedding-ada-002",
                      api_version =  "2023-03-15-preview",
                      api_key = open_ai_key)


In [9]:

# list of metrics we're going to use
metrics = [
    faithfulness,
    answer_relevancy,
    answer_similarity,
    answer_correctness,
    context_recall
]


result = evaluate(
    eval_data["eval"], metrics=metrics, llm=azure_openai_client, embeddings=azure_openai_embedding_client
)

result

Evaluating:   0%|          | 0/155 [00:00<?, ?it/s]

Evaluating:  30%|██▉       | 46/155 [01:23<03:41,  2.03s/it]Invalid JSON response. Expected dictionary with key 'Attributed'
Invalid JSON response. Expected dictionary with key 'Attributed'
Evaluating:  64%|██████▍   | 99/155 [03:17<01:52,  2.01s/it]Invalid JSON response. Expected dictionary with key 'Attributed'
Invalid JSON response. Expected dictionary with key 'Attributed'
Evaluating:  75%|███████▌  | 117/155 [03:42<01:22,  2.17s/it]Invalid JSON response. Expected dictionary with key 'Attributed'
Invalid JSON response. Expected dictionary with key 'Attributed'
Evaluating: 100%|██████████| 155/155 [05:41<00:00,  2.20s/it]
c:\Users\User\anaconda3\envs\test2env\lib\site-packages\ragas\evaluation.py:273: RuntimeWarning: Mean of empty slice
  value = np.nanmean(self.scores[cn])


{'faithfulness': 0.9667, 'answer_relevancy': 0.7511, 'answer_similarity': 0.9269, 'answer_correctness': 0.5502, 'context_recall': nan}

In [10]:
df = result.to_pandas()
df.head()

,question,ground_truth,contexts,answer,faithfulness,answer_relevancy,answer_similarity,answer_correctness,context_recall
0,How do we authenticate User licenses?,"User licenses, based on the customers Autodesk...",[\nFeature codes\n\nCascading sequences for Au...,User licenses for Autodesk products are authen...,1.000000,0.928129,0.978327,0.994582,NaN
1,How do we authenticate Standalone licenses?,Standalone licenses are tied to specific devic...,[\nFeature codes\n\nCascading sequences for Au...,Standalone licenses are authenticated by being...,1.000000,0.952950,0.995242,0.998811,NaN
2,How many types of licenses do adminstrators ma...,"Administrators manage three types of licenses,...",[\nFeature codes\n\nCascading sequences for Au...,"Administrators manage three types of licenses,...",1.000000,0.950667,0.980268,0.995067,NaN
3,What is Fusion 360 for Manufacturing?,Autodesk Fusion 360 for Manufacturing is an in...,[)\n\nManage Extension\n\nExecute change order...,Autodesk Fusion 360 for Manufacturing is an in...,1.000000,0.981863,0.991818,0.747955,NaN
4,What are Advanced CAM capabilities?,Unlock advanced tools for multi-axis CNC machi...,[\n\nExtend the core 3D design and manufacturi...,Advanced CAM capabilities in the context of Fu...,0.833333,0.941420,0.897434,0.724359,NaN


In [11]:
df

,question,ground_truth,contexts,answer,faithfulness,answer_relevancy,answer_similarity,answer_correctness,context_recall
0,How do we authenticate User licenses?,"User licenses, based on the customers Autodesk...",[\nFeature codes\n\nCascading sequences for Au...,User licenses for Autodesk products are authen...,1.000000,0.928129,0.978327,0.994582,NaN
1,How do we authenticate Standalone licenses?,Standalone licenses are tied to specific devic...,[\nFeature codes\n\nCascading sequences for Au...,Standalone licenses are authenticated by being...,1.000000,0.952950,0.995242,0.998811,NaN
2,How many types of licenses do adminstrators ma...,"Administrators manage three types of licenses,...",[\nFeature codes\n\nCascading sequences for Au...,"Administrators manage three types of licenses,...",1.000000,0.950667,0.980268,0.995067,NaN
3,What is Fusion 360 for Manufacturing?,Autodesk Fusion 360 for Manufacturing is an in...,[)\n\nManage Extension\n\nExecute change order...,Autodesk Fusion 360 for Manufacturing is an in...,1.000000,0.981863,0.991818,0.747955,NaN
4,What are Advanced CAM capabilities?,Unlock advanced tools for multi-axis CNC machi...,[\n\nExtend the core 3D design and manufacturi...,Advanced CAM capabilities in the context of Fu...,0.833333,0.941420,0.897434,0.724359,NaN
5,Who uses the Fusion 360 for Manufacturing offe...,Fusion 360 for Manufacturing is built for orga...,[)\n\nManage Extension\n\nExecute change order...,Fusion 360 for Manufacturing is built for orga...,1.000000,0.940401,0.995041,0.848760,NaN
6,How does Fusion 360 for Manufacturing differ t...,Autodesk offers a range of Fusion 360 extensio...,[ This often requires the production of a tool...,Fusion 360 for Manufacturing offers an integra...,1.000000,0.952221,0.976579,0.944145,NaN
7,Does Autodesk offer free software for students...,"If you are a student, educator, or an IT or la...",[ We recommend disabling pop-up blockers and t...,"Yes, Autodesk offers free one-year educational...",1.000000,0.952861,0.938279,0.834570,NaN
8,What are the Flexnet feature codes,Feature codes come in three types: product-spe...,"[\nFind a key, code, or cascading sequence\n\n...",The provided sources do not include specific i...,NaN,0.000000,0.878511,0.219628,NaN
9,What are the uses of Revit + AutoCAD + Civil 3D,"USE IT FOR: Building design, Infrastructure de...",[\n\nIn the package Top products Revit Civil 3...,"Revit, AutoCAD, and Civil 3D are used for vari...",1.000000,0.940585,0.886817,0.621704,NaN


In [12]:
df.to_csv("Evaluation_answers_hybrid.csv")